In [1]:
import plotly.graph_objects as go
import numpy as np
from dataclasses import dataclass
from typing import List, Dict
from pulp import *

from visualization.event_drawer import draw_daily_events
from visualization.calendar_drawer import draw_calendar
from data.calendar import Calendar
from data.event import Event, EmptyEvent

np.random.seed(1)


# User input


Users provides input with
* calendar setting
* Fixed events (randomly generated)
* Events to assign for 2 Calendars

MIP maximizes the objective value and produces daily assignment maximizing importance of scheduled events


### Notation

#### Sets
* $ i \in \mathcal{E}$ events
* $ i \in \mathcal{E}^f$ fixed events
* $ c \in \mathcal{C}$ calendars to fill
* $ d \in \mathcal{D}$ days
* $ t \in \mathcal{T}$ hourly time slots

#### Variables
* $z_{i,c,d,t}$ boolean variable, 1 if event $i$ was assigned to calendar $c$, day $d$ and hour $t$


#### Constants
* $K_i$ - duration of event $i$
* $F_{c,d,t} \leq 1$ - number of fixed events in calendar $c$, day $d$ and hour $t$
* $L_{c}$ - daily load (hours) allowed in each calendar

$ \max \sum_{i \in \mathcal{E}} \sum_{c \in \mathcal{C}} \sum_{d \in \mathcal{D}} \sum_{t \in \mathcal{T}} W_i z_{i,c,d,t}$

$s.t. $

$\sum_{i \in \mathcal{E}} z_{i,c,d,t} +\sum_{t' < t, t-t'<K_i -1} \sum_{i \in \mathcal{E}} z_{i,c,d,t'}
+ F_{c,d,t}
\leq 1 \;, \forall c \in \mathcal{C}, d \in \mathcal{D},  t \in \mathcal{T}$ [At maximum one event in each time slot]


$ \sum_{i \in \mathcal{E}} \sum_{t \in \mathcal{T}} z_{i,c,d,t} \leq L_c\;, \forall c \in \mathcal{C}, d \in \mathcal{D}$ [Daily load is respected]

$z_{i,c,d,t} \in \{0,1\}$

In [9]:
calendar = Calendar("mine",11,19)
assignment = {0:{15:Event("Barber",10,20,range(7),1,2,1)},
             1:{},
             2:{},
             3:{},
             4:{},
             5:{},
             6:{}}

In [11]:
fig = draw_calendar(calendar,assignment)
draw_calendar(calendar,assignment,fig,5)
fig.show()

In [12]:
def schedule_events(calendars: List[Calendar], events: List[Event], fixed_events: Dict[Calendar, List[Event]]): 
    pass

In [13]:
events = []
fixed_events = []
calendars = []
days = 7

In [14]:
calendar_timeslots = [range(c.day_start_time,c.day_end_time+1) for c in calendars]

In [15]:
from ortools.linear_solver import pywraplp


In [16]:
z = { (i,c,d,t): solver.IntVar(0, 1, f'z_{i}_{c}_{d}_{t}') 
                 if (d in event.possible_days) and (t<=event.max_time) and (t>=event.min_time)
                 else 0
     
                 for i,event in enumerate(events) 
                 for c,_ in enumerate(calendars) for d in range(days)  for t in calendar_timeslots[c]}


In [17]:
# one event per time slot constraint
for c,calendar in enumerate(calendars):
    for d in range(days):
        for t in calendar_timeslots[c]:
            number_fixed = len([e for e in fixed_events[calendar] if ((d in e.possible_days)
                              and (t >= e.min_time)
                              and (t <= e.max_time)) ] )                

            assert number_fixed <=1, "Cannot have more than 1 event assigned for the same time slot"
            
            if number_fixed == 0:
                assigned_now = sum(z[i, c, d ,t] for i,_ in enumerate(events))
                assigned_before = sum(z[i, c, d ,t2] for i,event in enumerate(events) 
                                      for t2 in range(calendar_timeslots[c]) if (t2<t) and (event.duration>t2-t) )
                solver.Add(assigned_now + assigned_before <= 1)
                
    

In [18]:
# daily load is respected 
for c,calendar in enumerate(calendars):
    for d in range(days):       
        
        fixed_load = len([t for e in fixed_events[calendar] if ((d in e.possible_days)
                              and (t >= e.min_time)
                              and (t <= e.max_time)) ] )            

        assert fixed_load <=calendar.daily_load, "Fixed load is above the daily load limit"

        assigned_load = sum(z[i, c, d ,t]*event.duration  for i,event in enumerate(events ) for t in calendar_timeslots[c])

        solver.Add(assigned_load + fixed_load <= calendar.daily_load)
                
    

In [19]:
# Event is assigned only # repetition times

for i,event in enumerate(events):
    solver.Add(sum(z[i,c,d,t]) for i,event in enumerate(events) 
                 for c,_ in enumerate(calendars) for d in range(days)  for t in calendar_timeslots[c]<= event.repetition)



In [20]:
solver = pywraplp.Solver.CreateSolver('SCIP')
infinity = solver.infinity()
# x and y are integer non-negative variables.
x = solver.IntVar(0.0, infinity, 'x')
y = solver.IntVar(0.0, infinity, 'y')

print('Number of variables =', solver.NumVariables())
# x + 7 * y <= 17.5.
solver.Add(x + 7 * y <= 17.5)

# x <= 3.5.
solver.Add(x <= 3.5)

print('Number of constraints =', solver.NumConstraints())

solver.Maximize(x + 10 * y)


Number of variables = 2
Number of constraints = 2


In [21]:
solver.Maximize(sum( z[i,c,d,t] * event.importance
    for i,event in enumerate(events) 
                 for c,_ in enumerate(calendars) for d in range(days)  for t in calendar_timeslots[c]))

In [22]:
status = solver.Solve()


In [23]:
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('x =', x.solution_value())
    print('y =', y.solution_value())
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 0.0
x = -0.0
y = -0.0
